In [2]:
import os
import pickle

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model as plot
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical
from keras.models import load_model

# Hyper params
MAXLEN = 100
MAX_FEATURES = 200000
BATCH_SIZE = 32
MAX_NB_WORDS=1000
EPOCHS=50

MODEL_ARCH_FOLDER = 'model_architecture/'
MODEL_CHECKPOINT_FOLDER = 'checkpoint/'
TOKENIZER = 'tokenizer.pkl'
MODEL_NAME = 'weights-improvement-.hdf5'
print('Hyper params set')
import os
os.chdir('C:\\Users\\Jameel shaik\\Documents\\Projects\\Personalized Medicine Redefining Cancer Treatment')
source= 'C:\\Users\\Jameel shaik\\Documents\\Projects\\Personalized Medicine Redefining Cancer Treatment'

#train_variant = pd.read_csv(source+"/training_variants")

# Load train
data_train = pd.read_csv(source+'/training_text', sep='\|\|', engine='python', 
                      skiprows=1, names=['ID', 'Text']).set_index('ID').reset_index()
#data_train=data_train.drop(['|'],axis=1)
print('Training Data Loaded')

# Load variants
data_training_variants = pd.read_csv(source+'/training_variants')

print('Variants loaded')


Using TensorFlow backend.


Hyper params set
Training Data Loaded
Variants loaded


In [3]:
!pip install keras

In [4]:
data_training_variants.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [5]:

# Merge X and Y
data_train_merge = pd.merge(data_train,data_training_variants,how='inner',on='ID')
print('Data merged')

# Dense layer can be encoded to 9. Add 1 to prediction class
data_train_merge['Class'] = data_train_merge['Class']-1
print('output variable differenced')

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

print('Tokenizing...')
tokenizer.fit_on_texts(data_train_merge['Text'])


print('Converting text to sequences...')
sequences_train = tokenizer.texts_to_sequences(data_train_merge['Text'])

word_index = tokenizer.word_index

print('Preparing data...')
x = sequence.pad_sequences(sequences_train, maxlen=MAXLEN)
y = np.array(data_train_merge['Class'])

y_binary = to_categorical(y)

print('Split train and test...')
rng = np.random.RandomState(42)
n_samples = len(x)
indices = np.arange(n_samples)
rng.shuffle(indices)
x_shuffled = x[indices]
y_shuffled = y[indices]

x_train = x_shuffled[:int(n_samples*0.8)]
x_test = x_shuffled[int(n_samples*0.8):]

y_train = y_shuffled[:int(n_samples*0.8)]
y_test = y_shuffled[int(n_samples*0.8):]

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print('Build model...')
model = Sequential()
model.add(Embedding(MAX_FEATURES, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(9, activation='sigmoid'))

model_pretrained_path = MODEL_CHECKPOINT_FOLDER + MODEL_NAME
if os.path.exists(model_pretrained_path):
    model = load_model(model_pretrained_path)
else:
    # try using different optimizers and different optimizer configs
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    json_string = model.to_json()
    
print('Train...')

model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(x_test, y_test))

Data merged
output variable differenced
Tokenizing...
Converting text to sequences...
Preparing data...
Split train and test...
Build model...
Train...
Train on 111 samples, validate on 28 samples
Epoch 1/50
111/111 [==============================] - 14s - loss: 2.1918 - acc: 0.2613 - val_loss: 2.1834 - val_acc: 0.3571
Epoch 2/50
111/111 [==============================] - 4s - loss: 2.1631 - acc: 0.4685 - val_loss: 2.1569 - val_acc: 0.3214
Epoch 3/50
111/111 [==============================] - 4s - loss: 2.0951 - acc: 0.3964 - val_loss: 2.0432 - val_acc: 0.3214
Epoch 4/50
111/111 [==============================] - 4s - loss: 1.7891 - acc: 0.3964 - val_loss: 1.9861 - val_acc: 0.3214
Epoch 5/50
111/111 [==============================] - 4s - loss: 1.6305 - acc: 0.3964 - val_loss: 2.0161 - val_acc: 0.3214
Epoch 6/50
111/111 [==============================] - 4s - loss: 1.5828 - acc: 0.3964 - val_loss: 2.0515 - val_acc: 0.3214
Epoch 7/50
111/111 [==============================] - 4s - loss: